In [1]:
""" First attempt to build a ConvNet in Keras
	for CV (Computer Vision), more specifically for 
	Image Recognition (classification problem).
"""

# Import libraries
import logging
import numpy as np
import keras.backend as K 
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

# For reproducibility
np.random.seed(42)

Using TensorFlow backend.


In [2]:
"""Retrieve the CIFAR dataset and process the data."""

nb_classes = 10
input_dim = 32*32*3
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

print("Cifar 10 data retrieved and processed")

Cifar 10 data retrieved and processed


In [3]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def leaky(x):
    return K.maximum(0.3*x, x)

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [4]:
def accuracy(y_pred, y_test):
    y_hat = np.argmax(y_pred, axis=1)
    y = np.argmax(y_test, axis=1)

    good = np.sum(np.equal(y, y_hat))
    return float(good/len(y_test))

In [5]:
activations = [e_swish_2, swish, "relu", "elu", leaky]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_2', 'swish', 'relu', 'elu', 'leaky']


In [8]:
def create(act):
    
    model = Sequential()
    # First Convolutional Block
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation=act, input_shape=x_train.shape[1:]))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation=act))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    # Second Convolutional Block
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation=act))
    model.add(Dropout(0.15))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation=act))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    # Last Convolutional Block
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation=act))
    model.add(MaxPooling2D(pool_size = 2))
    model.add(Dropout(0.15))
    # Flatten and pass trough fully connected layers
    model.add(Flatten())
    # Classification task
    model.add(Dense(64, activation=act))
    model.add(Dense(10, activation='softmax'))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])
        
    return model

In [6]:
# Record settings
LOG_FORMAT = "%(levelname)s %(asctime)s - %(message)s"
logging.basicConfig(filename="cifar10_simple_cnn_3_log.txt",format = LOG_FORMAT, level = logging.DEBUG, filemode = "a")
logs = logging.getLogger()

In [ ]:
import gc
for i,act in enumerate(activations):
    logs.info("\n \n Starting training for "+names[i])
    ensembler = 0
    logger = [names[i]]
    for i in range(3):
        gc.collect()
        model = create(act)
        his = model.fit(x_train, y_train, epochs=15, batch_size=64*2,  verbose=1, validation_data=(x_test, y_test))
        # Record accuracy of each model and save it
        logger.append(model.evaluate(x_test, y_test)[1])
        # Calculate probabilities of test data and sum them toghether
        ensembler += model.predict_proba(x_test)
        # Clear session (GPU MEMORY)
        K.get_session().close()
        K.set_session(K.tf.Session())
        del model, his
        
    # Calculate the median accuracy
    ensembled = accuracy(ensembler, y_test)
    print(ensembled)
    logs.info(str(logger))
    logs.info(str(ensembled))
    del ensembler, ensembled

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 77s 2ms/step - loss: 1.5750 - acc: 0.4259 - val_loss: 1.4423 - val_acc: 0.4861
Epoch 2/15
50000/50000 [==============================] - 74s 1ms/step - loss: 1.1439 - acc: 0.5893 - val_loss: 1.1077 - val_acc: 0.6065
Epoch 3/15
50000/50000 [==============================] - 74s 1ms/step - loss: 0.9756 - acc: 0.6537 - val_loss: 1.0490 - val_acc: 0.6359
Epoch 4/15
50000/50000 [==============================] - 74s 1ms/step - loss: 0.8594 - acc: 0.6958 - val_loss: 0.8763 - val_acc: 0.6917
Epoch 5/15
50000/50000 [==============================] - 75s 1ms/step - loss: 0.7729 - acc: 0.7284 - val_loss: 0.7720 - val_acc: 0.7306
Epoch 6/15
50000/50000 [==============================] - 75s 1ms/step - loss: 0.6971 - acc: 0.7549 - val_loss: 0.7178 - val_acc: 0.7501
Epoch 7/15
50000/50000 [==============================] - 74s 1ms/step - loss: 0.6416 - acc: 0.7736 - val_loss: 0.6996 - val_acc

Epoch 13/15
50000/50000 [==============================] - 61s 1ms/step - loss: 0.4323 - acc: 0.8456 - val_loss: 0.6488 - val_acc: 0.7833
Epoch 14/15
50000/50000 [==============================] - 61s 1ms/step - loss: 0.4087 - acc: 0.8547 - val_loss: 0.6260 - val_acc: 0.7943
Epoch 15/15
10000/10000 [==============================] - 4s 405us/step
Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 62s 1ms/step - loss: 1.5651 - acc: 0.4306 - val_loss: 1.3009 - val_acc: 0.5248
Epoch 2/15
50000/50000 [==============================] - 61s 1ms/step - loss: 1.1786 - acc: 0.5781 - val_loss: 1.0869 - val_acc: 0.6149
Epoch 3/15
50000/50000 [==============================] - 61s 1ms/step - loss: 1.0087 - acc: 0.6406 - val_loss: 0.9599 - val_acc: 0.6606
Epoch 4/15
50000/50000 [==============================] - 61s 1ms/step - loss: 0.8843 - acc: 0.6865 - val_loss: 0.8917 - val_acc: 0.6848
Epoch 5/15
50000/50000 [=============================

50000/50000 [==============================] - 57s 1ms/step - loss: 0.7349 - acc: 0.7413 - val_loss: 0.7919 - val_acc: 0.7334
Epoch 6/15
50000/50000 [==============================] - 57s 1ms/step - loss: 0.6760 - acc: 0.7620 - val_loss: 0.7213 - val_acc: 0.7525
Epoch 7/15
50000/50000 [==============================] - 57s 1ms/step - loss: 0.6284 - acc: 0.7783 - val_loss: 0.6854 - val_acc: 0.7686
Epoch 8/15
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5876 - acc: 0.7937 - val_loss: 0.6820 - val_acc: 0.7689
Epoch 9/15
50000/50000 [==============================] - 57s 1ms/step - loss: 0.5474 - acc: 0.8070 - val_loss: 0.6769 - val_acc: 0.7746
Epoch 10/15
50000/50000 [==============================] - 56s 1ms/step - loss: 0.5230 - acc: 0.8160 - val_loss: 0.6633 - val_acc: 0.7821
Epoch 11/15
50000/50000 [==============================] - 56s 1ms/step - loss: 0.4966 - acc: 0.8257 - val_loss: 0.6709 - val_acc: 0.7768
Epoch 12/15
50000/50000 [=========================

In [9]:
import gc
for i,act in enumerate(activations):
    if i < 4: pass
    else:
        logs.info("\n \n Starting training for "+names[i])
        ensembler = 0
        logger = [names[i]]
        for i in range(3):
            gc.collect()
            model = create(act)
            his = model.fit(x_train, y_train, epochs=15, batch_size=64*2,  verbose=1, validation_data=(x_test, y_test))
            # Record accuracy of each model and save it
            logger.append(model.evaluate(x_test, y_test)[1])
            # Calculate probabilities of test data and sum them toghether
            ensembler += model.predict_proba(x_test)
            # Clear session (GPU MEMORY)
            K.get_session().close()
            K.set_session(K.tf.Session())
            del model, his

        # Calculate the median accuracy
        ensembled = accuracy(ensembler, y_test)
        print(ensembled)
        logs.info(str(logger))
        logs.info(str(ensembled))
        del ensembler, ensembled

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 69s 1ms/step - loss: 1.5201 - acc: 0.4460 - val_loss: 1.2909 - val_acc: 0.5379
Epoch 2/15
50000/50000 [==============================] - 63s 1ms/step - loss: 1.1186 - acc: 0.6027 - val_loss: 1.1061 - val_acc: 0.6123
Epoch 3/15
50000/50000 [==============================] - 63s 1ms/step - loss: 0.9513 - acc: 0.6653 - val_loss: 1.0620 - val_acc: 0.6333
Epoch 4/15
50000/50000 [==============================] - 63s 1ms/step - loss: 0.8434 - acc: 0.7046 - val_loss: 0.8630 - val_acc: 0.6982
Epoch 5/15
50000/50000 [==============================] - 63s 1ms/step - loss: 0.7667 - acc: 0.7334 - val_loss: 0.7600 - val_acc: 0.7361
Epoch 6/15
50000/50000 [==============================] - 62s 1ms/step - loss: 0.6992 - acc: 0.7562 - val_loss: 0.7310 - val_acc: 0.7442
Epoch 7/15
50000/50000 [==============================] - 63s 1ms/step - loss: 0.6507 - acc: 0.7731 - val_loss: 0.7168 - val_acc

In [ ]:
# LOGS CAN BE FOUND AT THE cifar_10_small_3_logs.txt FILE